<h1 id="header">BookMe Company</h1>

### Supervised Project

In [8]:
# Relevant Package Importing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

### Pre-Processing
Need to solve nulls in year of birth

##### Should we change the nulls of year here or after spliting the data?

In [9]:
#Import Data from cluster.csv
db = pd.read_csv('train.csv')
db[db.duplicated(keep = False)] # We don't have duplicates
# Fill year of brith NAs with median
db['Year_Birth'] = db['Year_Birth'].fillna(db['Year_Birth'].median()).astype(int).astype(np.int64)

In [10]:
test = pd.read_csv('test.csv')

##### Replace incorrect performance values

In [11]:
db['Wifi'] = db['Wifi'].replace(6,5)
# db['Comfort'] = db['Comfort'].replace(0,1)
# db['ReceptionSchedule'] = db['ReceptionSchedule'].replace(0,1)
# db['FoodDrink'] = db['FoodDrink'].replace(0,1)
# db['Wifi'] = db['Wifi'].replace(0,1).replace(6,5)
# db['Amenities'] = db['Amenities'].replace(0,1)
# db['OnlineBooking'] = db['OnlineBooking'].replace(0,1)
# db['RoomSpace'] = db['RoomSpace'].replace(0,1)
# db['BarService'] = db['BarService'].replace(0,1)

In [12]:
# test['Comfort'] = test['Comfort'].replace(0,1)
# test['ReceptionSchedule'] = test['ReceptionSchedule'].replace(0,1)
# test['FoodDrink'] = test['FoodDrink'].replace(0,1)
# test['Location'] = test['Location'].replace(0,1)
# test['Wifi'] = test['Wifi'].replace(0,1)
# test['Amenities'] = test['Amenities'].replace(0,1)
# test['OnlineBooking'] = test['OnlineBooking'].replace(0,1)
# test['RoomSpace'] = test['RoomSpace'].replace(0,1)
# test['BarService'] = test['BarService'].replace(0,1)

In [13]:
# Converting every column that is int64 to int8 for performance issues
db.iloc[:,(db.dtypes=='int64').values] = db.iloc[:,(db.dtypes=='int64').values].astype(np.int16)
test.iloc[:,(test.dtypes=='int64').values] = test.iloc[:,(test.dtypes=='int64').values].astype(np.int16)

##### Change Year of Birth with Date

In [14]:
# from datetime import date
# db['Age'] = date.today().year - db['Year_Birth']
# db.drop(columns = ['Year_Birth'], axis=1, inplace=True)
# test['Age'] = date.today().year - test['Year_Birth']
# test.drop(columns = ['Year_Birth'], axis=1, inplace=True)

##### Set Customer ID as Index

In [15]:
db.set_index('Cust_ID', inplace = True)

In [16]:
#db.describe().T

##### Notes from initial pre-processing
- We have a minimum age of 8 years and max of 81 in the test file
- Min 8 and max 86 in the train file

##### We should't make new variables, dummy variables, nor replace categorical values for numerical, since we need the end variables to be the same as the test variables

In [17]:
db['Longevity'] = db['Longevity'].replace('y','yes') #Yes and y both show as different values, will be changed into "yes"
# db['Longevity'] = db['Longevity'].replace('yes',1).replace('no',0)
# test['Longevity'] = test['Longevity'].replace('yes',1).replace('no',0)

In [18]:
# db['TypeTravel'] = db['TypeTravel'].replace('business',1).replace('leisure',0) #1 will represent business and 0 will represent leisure
# test['TypeTravel'] = test['TypeTravel'].replace('business',1).replace('leisure',0)

In [19]:
# gender = db['Name'].str.split(' ',expand = True,n=1)[0]
# db['Gender'] = gender
# db['Gender'] = db['Gender'].replace('Ms.',0).replace('Mr.',1)
# db.drop(columns = ['Name'], axis = 1, inplace = True)

# gender = test['Name'].str.split(' ',expand = True,n=1)[0]
# test['Gender'] = gender
# test['Gender'] = test['Gender'].replace('Ms.',0).replace('Mr.',1)
# test.drop(columns = ['Name'], axis = 1, inplace = True)

----
##### Start Division between Data and Target

In [20]:
data = db.drop(['Churn'], axis=1)
target = db['Churn']

In [21]:
#print(target.info())
#print(data.info())

##### If we want to use Hold-out method

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_val, y_train, y_val = train_test_split(data, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=15, 
                                                    shuffle=True, 
                                                    stratify=target
                                                   )

In [24]:
print('train:{}% | validation:{}%'.format(round(len(y_train)/len(target),2),
                                                     round(len(y_val)/len(target),2)
                                                    ))

train:0.8% | validation:0.2%


##### Dividing DataSet between Numerical and Categorical data

In [25]:
# Numerical data
X_train_num = X_train.select_dtypes(include=np.number).set_index(X_train.index)
X_val_num = X_val.select_dtypes(include=np.number).set_index(X_val.index)

# Categorical data
X_train_cat = X_train.select_dtypes(exclude=np.number).set_index(X_train.index)
X_val_cat = X_val.select_dtypes(exclude=np.number).set_index(X_val.index)

##### Scaling numerical data between 0 and 1

In [26]:
scaler = MinMaxScaler()
scaler.fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num) # this will return an array
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns, index = X_train.index) # Convert the array to a pandas dataframe
X_train_num_scaled.head(5)

,Year_Birth,RewardPoints,Comfort,ReceptionSchedule,FoodDrink,Location,Wifi,Amenities,Staff,OnlineBooking,PriceQuality,RoomSpace,CheckOut,Checkin,Cleanliness,BarService
Cust_ID,,,,,,,,,,,,,,,,
9587,0.846154,0.508026,0.6,0.4,0.6,0.50,0.8,0.8,0.75,0.8,0.75,0.8,1.00,0.50,0.75,0.8
4281,0.551282,0.695459,0.2,0.2,0.2,0.75,0.2,0.2,0.00,0.6,1.00,0.4,0.75,0.00,0.75,0.2
6256,0.576923,0.153340,0.6,0.6,0.6,0.50,0.6,0.2,0.00,0.6,1.00,0.8,0.75,0.00,1.00,0.2
10584,0.358974,0.941752,0.6,1.0,1.0,1.00,0.2,0.6,0.75,0.6,0.50,0.6,0.50,0.50,0.50,0.6
9339,0.384615,0.912399,1.0,1.0,1.0,1.00,1.0,1.0,0.75,1.0,1.00,1.0,1.00,0.75,1.00,0.6


In [27]:
X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns, index = X_val.index)
X_val_num_scaled.head(5)

,Year_Birth,RewardPoints,Comfort,ReceptionSchedule,FoodDrink,Location,Wifi,Amenities,Staff,OnlineBooking,PriceQuality,RoomSpace,CheckOut,Checkin,Cleanliness,BarService
Cust_ID,,,,,,,,,,,,,,,,
6798,0.346154,0.423024,0.6,0.8,0.8,0.75,0.6,0.6,0.50,0.6,0.50,0.6,0.50,0.25,0.50,0.6
7661,0.564103,0.753096,0.4,0.4,0.4,0.25,0.8,1.0,1.00,0.8,0.75,0.8,0.75,0.75,0.75,0.6
5965,0.602564,0.735514,0.6,0.6,0.4,0.50,1.0,0.8,1.00,0.6,0.50,0.6,0.50,0.50,0.50,1.0
1171,0.974359,0.689344,0.2,0.8,0.2,0.50,0.4,0.2,0.25,0.4,1.00,0.8,0.75,0.75,0.75,0.4
11840,0.756410,0.604953,0.4,0.4,0.4,0.75,0.8,0.4,0.75,0.8,0.50,1.0,0.75,0.75,1.00,0.8


### Feature Selection

----

#### Apply Logistic Regression

In [77]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

In [ ]:
print('Train:', log_model.score(X_train, y_train))
print('Validation:', log_model.score(X_val, y_val))

----

##### K-Fold Cross Validation

In [ ]:
from sklearn.linear_model import LogisticRegression